In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import glob,sys,os,pickle,gc
from tqdm import tqdm

In [2]:
in_parquet = "/home/antslab/data_hdd4t/pcap_process/pcap_spark/2020_01_10/中華電信/parquet/2020_01_10_session_v5.parquet"
df = pd.read_parquet(in_parquet)
pickle_dir = '/home/antslab/spark_data/pcap_inter/2020_01_10/中華電信/case_pickles/'
df.columns

Index(['frame_time', 'frame_time_list', 'session_duration', 'session_packets',
       'tcp_window_size_value_avg', 'tcp_window_size_value_max',
       'tcp_payload_size', 'tcp_hdr_len_sum', 'tcp_srcport', 'tcp_dstport',
       'tcp_flags_ack', 'tcp_flags_fin', 'tcp_flags_push', 'tcp_flags_reset',
       'tcp_flags_syn', 'ip_src', 'ip_dst', 'ip_ttl_avg', 'ip_ttl_max',
       'ip_proto', 'frame_protocols_common', 'frame_protocols_max',
       'frame_len_sum', 'country', 'city', 'subdivisions', 'latitude',
       'longitude', 'domain', 'isp', 'network', 'udp_length_sum',
       'udp_srcport', 'udp_dstport'],
      dtype='object')

In [3]:
df['tcp_payload_avg'] = df.tcp_payload_size/df.session_packets
df['udp_payload_size'] = df.udp_length_sum - (df.session_packets*8)
df['udp_payload_avg'] = df.udp_payload_size/df.session_packets

### case11
* protocol max include "eth:ethertype:ip:tcp:nbss:smb"
* filter: session_duration、session_packets、tcp_payload_avg、tcp packet size

In [11]:
df_smb = df[df['frame_protocols_max'].str.contains("eth:ethertype:ip:tcp:nbss:smb")]
df_smb

,frame_time,frame_time_list,session_duration,session_packets,tcp_window_size_value_avg,tcp_window_size_value_max,tcp_payload_size,tcp_hdr_len_sum,tcp_srcport,tcp_dstport,tcp_flags_ack,tcp_flags_fin,tcp_flags_push,tcp_flags_reset,tcp_flags_syn,ip_src,ip_dst,ip_ttl_avg,ip_ttl_max,ip_proto,frame_protocols_common,frame_protocols_max,frame_len_sum,country,city,subdivisions,latitude,longitude,domain,isp,network,udp_length_sum,udp_srcport,udp_dstport,tcp_payload_avg,udp_payload_size,udp_payload_avg
464,1.578615e+09,"[1578614895.674232, 1578614895.674513, 1578614...",1051.190186,833,16590.553421,31088.0,17598.0,20060.0,37843.0,445.0,566.0,108.0,159.0,46.0,357.0,103.80.142.3,210.242.149.112,98.677071,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,66488,Philippines,Zamboanga City,Province of Zamboanga del Sur,6.9103,122.0739,None,Department of Science and Technology,103.80.142.0/23,NaN,NaN,NaN,21.126050,NaN,NaN
465,1.578616e+09,"[1578616066.907181, 1578616067.006687, 1578616...",5.399179,44,23580.522727,31088.0,1415.0,928.0,8274.0,445.0,41.0,6.0,15.0,0.0,6.0,103.80.142.3,210.242.149.65,85.181818,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,3923,Philippines,Zamboanga City,Province of Zamboanga del Sur,6.9103,122.0739,None,Department of Science and Technology,103.80.142.0/23,NaN,NaN,NaN,32.159091,NaN,NaN
466,1.578616e+09,"[1578616192.32045, 1578616192.421074, 15786161...",267.281677,213,23263.469484,31088.0,6284.0,4548.0,63134.0,445.0,195.0,28.0,65.0,4.0,36.0,103.80.142.3,210.242.149.66,85.901408,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,18470,Philippines,Zamboanga City,Province of Zamboanga del Sur,6.9103,122.0739,None,Department of Science and Technology,103.80.142.0/23,NaN,NaN,NaN,29.502347,NaN,NaN
467,1.578617e+09,"[1578616579.658287, 1578616579.757794, 1578616...",379.252899,164,23369.506098,31088.0,4978.0,3472.0,8741.0,445.0,152.0,23.0,53.0,1.0,24.0,103.80.142.3,210.242.149.72,85.603659,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,14338,Philippines,Zamboanga City,Province of Zamboanga del Sur,6.9103,122.0739,None,Department of Science and Technology,103.80.142.0/23,NaN,NaN,NaN,30.353659,NaN,NaN
468,1.578617e+09,"[1578617078.912472, 1578617079.013061, 1578617...",9.458981,74,23268.594595,31088.0,2322.0,1576.0,26899.0,445.0,68.0,10.0,24.0,1.0,12.0,103.80.142.3,210.242.149.76,86.054054,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,6546,Philippines,Zamboanga City,Province of Zamboanga del Sur,6.9103,122.0739,None,Department of Science and Technology,103.80.142.0/23,NaN,NaN,NaN,31.378378,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3657326,1.578668e+09,"[1578668386.246967, 1578668386.247324, 1578668...",989.484375,696,17108.889368,64240.0,12757.0,17532.0,56507.0,445.0,483.0,106.0,113.0,51.0,269.0,95.181.2.147,210.242.146.64,93.080460,108,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb:smb:smb,54647,Russia,Kemerovo,Kemerovo Oblast,55.3333,86.0833,goodline.info,E-Light-Telecom Ltd.,95.181.0.0/17,NaN,NaN,NaN,18.329023,NaN,NaN
3657327,1.578670e+09,"[1578669501.021697, 1578669501.396471, 1578669...",135.747482,83,9521.722892,29200.0,2830.0,1804.0,56444.0,445.0,77.0,12.0,30.0,0.0,12.0,95.181.2.147,210.242.146.72,85.204819,108,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,7594,Russia,Kemerovo,Kemerovo Oblast,55.3333,86.0833,goodline.info,E-Light-Telecom Ltd.,95.181.0.0/17,NaN,NaN,NaN,34.096386,NaN,NaN
3657328,1.578670e+09,"[1578669756.851886, 1578669757.212345, 1578669...",16.079573,44,8994.272727,29200.0,1415.0,952.0,59583.0,445.0,41.0,6.0,15.0,0.0,6.0,95.181.2.147,210.242.146.74,84.000000,108,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,3947,Russia,Kemerovo,Kemerovo Oblast,55.3333,86.0833,goodline.info,E-Light-Telecom Ltd.,95.181.0.0/17,NaN,NaN,NaN,32.159091,NaN,NaN
3657329,1.578670e+09,"[1578669893.001262, 1578669893.366252, 1578669...",136.030350,75

In [12]:
sd_q3 = df_smb.session_duration.quantile(0.75)
sd_iqr = sd_q3-df_smb.session_duration.quantile(0.25)

sp_q3 = df_smb.session_packets.quantile(0.75)
sp_iqr = sp_q3 - df_smb.session_packets.quantile(0.25)

tpa_q3 =  df_smb.tcp_payload_avg.quantile(0.75)
tpq_iqr = tpa_q3 - df_smb.tcp_payload_avg.quantile(0.25)

tps_q3 = df_smb.tcp_payload_size.quantile(0.75)
tps_iqr = tps_q3 - df_smb.tcp_payload_size.quantile(0.25)

df_filter1_smb = df_smb[df_smb.session_duration>sd_q3+3*sd_iqr]
df_filter2_smb = df_filter1_smb[df_filter1_smb.session_packets>sp_q3+3*sp_iqr]
df_filter3_smb = df_filter2_smb[df_filter2_smb.tcp_payload_avg>tpa_q3+3*tpq_iqr]
df_filter4_smb = df_filter3_smb[df_filter3_smb.tcp_payload_size>tps_q3+3*tps_iqr]
df_filter4_smb

,frame_time,frame_time_list,session_duration,session_packets,tcp_window_size_value_avg,tcp_window_size_value_max,tcp_payload_size,tcp_hdr_len_sum,tcp_srcport,tcp_dstport,tcp_flags_ack,tcp_flags_fin,tcp_flags_push,tcp_flags_reset,tcp_flags_syn,ip_src,ip_dst,ip_ttl_avg,ip_ttl_max,ip_proto,frame_protocols_common,frame_protocols_max,frame_len_sum,country,city,subdivisions,latitude,longitude,domain,isp,network,udp_length_sum,udp_srcport,udp_dstport,tcp_payload_avg,udp_payload_size,udp_payload_avg
18391,1.578618e+09,"[1578618033.719293, 1578618033.723177, 1578618...",1010.093689,730,29781.923288,65535.0,80888.0,18052.0,54482.0,445.0,567.0,87.0,189.0,32.0,245.0,113.189.231.122,210.242.148.83,99.063014,114,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,124338,Vietnam,Hanoi,Hanoi,21.0313,105.8516,vnpt.vn,Vietnam Posts And Telecommunications Group,113.188.0.0/15,NaN,NaN,NaN,110.805479,NaN,NaN
18521,1.578625e+09,"[1578624791.862712, 1578624791.862892, 1578624...",619.007507,446,10319.248879,29200.0,84225.0,10500.0,53851.0,445.0,341.0,35.0,140.0,17.0,123.0,117.6.225.134,210.242.147.71,95.746637,112,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,110449,Vietnam,Hoang Mai,Tinh Bac Giang,21.2500,106.1167,None,Viettel Group,117.6.0.0/15,NaN,NaN,NaN,188.845291,NaN,NaN
30448,1.578592e+09,"[1578592067.718862, 1578592067.719193, 1578592...",1080.856445,643,26358.211509,65535.0,72845.0,16200.0,55185.0,445.0,450.0,85.0,130.0,27.0,249.0,58.27.253.2,210.242.148.253,97.752722,113,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,111491,Pakistan,Karachi,Sindh,24.9043,67.0817,wateen.net,Wateen Telecom,58.27.128.0/17,NaN,NaN,NaN,113.289269,NaN,NaN
71914,1.578648e+09,"[1578647857.413633, 1578647857.414027, 1578647...",558.807251,378,21095.195767,64308.0,71079.0,8672.0,57314.0,445.0,313.0,39.0,91.0,15.0,86.0,113.176.100.12,210.242.146.132,95.891534,114,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,93131,Vietnam,Da Nang,Da Nang,16.0685,108.2215,None,Vietnam Posts And Telecommunications Group,113.176.0.0/16,NaN,NaN,NaN,188.039683,NaN,NaN
75441,1.578607e+09,"[1578607304.511732, 1578607304.512025, 1578607...",854.864563,884,14233.123303,65340.0,76090.0,22556.0,20994.0,445.0,627.0,83.0,157.0,66.0,363.0,190.214.121.100,210.242.145.199,97.990950,112,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,129366,Ecuador,Babahoyo,Provincia de Los Rios,-1.8167,-79.5167,cnt-grms.ec,Corporacion Nacional De Telecomunicaciones - C...,190.214.120.0/22,NaN,NaN,NaN,86.074661,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3634475,1.578661e+09,"[1578661026.924741, 1578661030.200215, 1578661...",875.784241,4128,48305.894380,65535.0,1443668.0,89416.0,2828.0,445.0,3414.0,248.0,1357.0,584.0,859.0,46.24.70.178,210.242.148.60,88.422238,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb:ntlmssp,1680326,Spain,Cartagena,Murcia,37.6051,-0.9862,comunitel.net,Vodafone Spain,46.24.64.0/21,NaN,NaN,NaN,349.725775,NaN,NaN
3634479,1.578665e+09,"[1578664542.968962, 1578664546.241712, 1578664...",739.705688,3215,47612.312908,65535.0,1056830.0,69964.0,4244.0,445.0,2632.0,187.0,981.0,482.0,708.0,46.24.70.178,210.242.146.58,87.770451,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb:ntlmssp,1241984,Spain,Cartagena,Murcia,37.6051,-0.9862,comunitel.net,Vodafone Spain,46.24.64.0/21,NaN,NaN,NaN,328.718507,NaN,NaN
3634484,1.578669e+09,"[1578669320.780608, 1578669320.780815, 1578669...",2080.954102,8108,53265.399975,65535.0,4538193.0,165252.0,1562.0,445.0,7840.0,272.0,3907.0,100.0,389.0,46.24.70.178,210.242.147.39,84.879872,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb:gss-api:ntlmssp,4993629,Spain,Cartagena,Murcia,37.6051,-0.9862,comunitel.net,Vodafone Spain,46.24.64.0/21,NaN,NaN,NaN,559.717933,NaN,NaN
3634519,1.578660e+09,"[1578659767.926574, 1578659767.926801, 1578659...",674.812683,614,40675.361564,65392.0,71981.0,14348.0,59522.0,445.0,468.0,84.0

In [13]:
df_filter4_smb.domain.value_counts().head(20)

vnpt.vn                  30
comunitel.net            22
hinet.net                11
viettel.vn               10
prod-infinitum.com.mx     6
pldt.net                  6
otenet.gr                 6
actcorp.in                6
totinternet.net           5
rr.com                    4
uninet-ide.com.mx         4
rima-tde.net              4
airtelbroadband.in        3
megared.net.mx            3
myaisfibre.com            3
cantv.net                 3
tfn.net.tw                3
wateen.net                2
bezeqint.net              2
com.co                    2
Name: domain, dtype: int64

In [24]:
len(df_filter4_smb[df_filter4_smb.domain == 'pldt.net'].city.value_counts())

4

In [82]:
df_filter4_smb.session_packets.sum()

417203

In [9]:
print(df_filter4_smb.domain.value_counts())
print(df_filter4_smb.network.value_counts())
df_filter4_smb.ip_src.value_counts()

vnpt.vn               30
comunitel.net         22
hinet.net             11
viettel.vn            10
otenet.gr              6
                      ..
182-airtel.com         1
vox.co.za              1
moratelindo.net.id     1
cmcti.vn               1
alshamil.net.ae        1
Name: domain, Length: 74, dtype: int64
46.24.64.0/21       21
160.120.0.0/16       8
94.68.0.0/15         6
115.72.0.0/13        5
14.162.0.0/15        5
                    ..
187.170.0.0/15       1
200.233.56.0/22      1
202.191.128.0/19     1
187.56.0.0/15        1
168.90.12.0/23       1
Name: network, Length: 213, dtype: int64


46.24.70.178       21
160.120.167.126     8
94.69.102.19        6
2.136.208.15        4
124.12.54.226       3
                   ..
185.100.87.250      1
113.179.40.208      1
171.231.96.147      1
201.164.65.178      1
78.29.28.166        1
Name: ip_src, Length: 251, dtype: int64

In [17]:
ips = df_filter4_smb['ip_dst'].value_counts().index#.head(60)
values = df_filter4_smb['ip_dst'].value_counts().values
a = 0
d = 0
g = 0
c = 0
for ip,value in zip(ips,values):
    value_ = int(ip.split('.')[-1])
    if value_<=63:
        a =a+value_
    elif value_ <=127:
        d = d + value_
    elif value_ <=191:
        g =g+value_
    elif value_ <=255:
        c = c+value_
    else:
        print(ip,value)

In [86]:
df_filter4_smb[df_filter4_smb.domain =='vnpt.vn'].country.value_counts()

Vietnam    30
Name: country, dtype: int64

In [26]:
df_filter4_smb.domain.value_counts().head(60)

vnpt.vn                      30
comunitel.net                22
hinet.net                    11
viettel.vn                   10
otenet.gr                     6
pldt.net                      6
prod-infinitum.com.mx         6
actcorp.in                    6
totinternet.net               5
uninet-ide.com.mx             4
rima-tde.net                  4
rr.com                        4
airtelbroadband.in            3
cantv.net                     3
megared.net.mx                3
tfn.net.tw                    3
myaisfibre.com                3
tedata.net                    2
bezeqint.net                  2
corbina.ru                    2
cnt-grms.ec                   2
in.th                         2
com.co                        2
wateen.net                    2
indosat.com                   1
copaco.com.py                 1
hypernettelecom.net.br        1
brasiltelecom.net.br          1
frontiernet.net               1
cotas.com.bo                  1
movistar.cl                   1
ttnet.co

In [84]:
df_filter5_smb = df_filter4_smb[df_filter4_smb.ip_src =='46.24.70.178']
df_filter5_smb

21

In [ ]:
case11_ips = df_filter5_smb.frame_time_list.tolist()
case11_ips = [list(x) for x in case11_ips]
pickle.dump(obj=case11_ips,file=open(pickle_dir+'case11.pkl','wb'))

payload

In [ ]:
case11 = pickle.load(open(pickle_dir+'case11_payload.pkl','rb'))
case11

In [27]:
df_filter51_smb = df_filter4_smb[df_filter4_smb.domain =='vnpt.vn']
df_filter51_smb

,frame_time,frame_time_list,session_duration,session_packets,tcp_window_size_value_avg,tcp_window_size_value_max,tcp_payload_size,tcp_hdr_len_sum,tcp_srcport,tcp_dstport,tcp_flags_ack,tcp_flags_fin,tcp_flags_push,tcp_flags_reset,tcp_flags_syn,ip_src,ip_dst,ip_ttl_avg,ip_ttl_max,ip_proto,frame_protocols_common,frame_protocols_max,frame_len_sum,country,city,subdivisions,latitude,longitude,domain,isp,network,udp_length_sum,udp_srcport,udp_dstport,tcp_payload_avg,udp_payload_size,udp_payload_avg
18391,1.578618e+09,"[1578618033.719293, 1578618033.723177, 1578618...",1010.093689,730,29781.923288,65535.0,80888.0,18052.0,54482.0,445.0,567.0,87.0,189.0,32.0,245.0,113.189.231.122,210.242.148.83,99.063014,114,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,124338,Vietnam,Hanoi,Hanoi,21.0313,105.8516,vnpt.vn,Vietnam Posts And Telecommunications Group,113.188.0.0/15,NaN,NaN,NaN,110.805479,NaN,NaN
219191,1.578619e+09,"[1578619494.98996, 1578619494.990135, 15786194...",630.065308,446,19386.280269,65535.0,70865.0,10952.0,60771.0,445.0,336.0,52.0,112.0,14.0,149.0,14.243.54.9,210.242.148.125,54.147982,64,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,97473,Vietnam,Da Nang,Da Nang,16.0685,108.2215,vnpt.vn,Vietnam Posts And Telecommunications Group,14.240.0.0/13,NaN,NaN,NaN,158.890135,NaN,NaN
290476,1.578618e+09,"[1578617906.922764, 1578617906.92308, 15786179...",900.265991,593,29272.753794,65535.0,76733.0,14404.0,59798.0,445.0,441.0,75.0,158.0,28.0,202.0,14.174.2.36,210.242.149.86,100.833052,115,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,111801,Vietnam,Dong Hoi,Tinh Quang Binh,17.4833,106.6000,vnpt.vn,Vietnam Posts And Telecommunications Group,14.168.0.0/13,NaN,NaN,NaN,129.397976,NaN,NaN
490343,1.578620e+09,"[1578619740.602898, 1578619740.603101, 1578619...",1672.032227,974,22692.203285,65535.0,87230.0,23116.0,48551.0,445.0,743.0,107.0,259.0,58.0,305.0,14.162.185.34,210.242.147.113,96.366530,113,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb:smb:smb,144624,Vietnam,Hanoi,Hanoi,21.0313,105.8516,vnpt.vn,Vietnam Posts And Telecommunications Group,14.162.0.0/15,NaN,NaN,NaN,89.558522,NaN,NaN
490419,1.578598e+09,"[1578597877.276446, 1578597877.27677, 15785978...",557.576843,434,17491.831797,65535.0,68365.0,11216.0,54247.0,445.0,294.0,30.0,85.0,21.0,210.0,14.226.84.70,210.242.147.166,95.108295,114,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,94783,Vietnam,Thanh Hóa,Tinh Thanh Hoa,19.8000,105.7667,vnpt.vn,Vietnam Posts And Telecommunications Group,14.226.0.0/15,NaN,NaN,NaN,157.523041,NaN,NaN
560798,1.578592e+09,"[1578592374.353669, 1578592389.363054, 1578592...",801.268250,485,18424.577320,64800.0,74946.0,11816.0,56316.0,445.0,369.0,40.0,138.0,13.0,157.0,14.188.20.139,210.242.146.36,97.740206,115,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,103788,Vietnam,Haiphong,Haiphong,20.8561,106.6822,vnpt.vn,Vietnam Posts And Telecommunications Group,14.184.0.0/13,NaN,NaN,NaN,154.527835,NaN,NaN
596136,1.578649e+09,"[1578649485.896912, 1578649485.897049, 1578649...",620.572998,509,19407.687623,65535.0,73067.0,12340.0,65207.0,445.0,374.0,66.0,130.0,21.0,168.0,14.232.244.254,210.242.145.125,98.288802,113,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,103247,Vietnam,Hanoi,Hanoi,21.0313,105.8516,vnpt.vn,Vietnam Posts And Telecommunications Group,14.232.0.0/14,NaN,NaN,NaN,143.550098,NaN,NaN
632304,1.578617e+09,"[1578617395.967146, 1578617482.418639, 1578617...",736.927917,513,19635.177388,64952.0,71970.0,12496.0,50411.0,445.0,386.0,65.0,129.0,18.0,170.0,14.162.145.3,210.242.145.59,97.988304,113,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,102438,Vietnam,Hanoi,Hanoi,21.0313,105.8516,vnpt.vn,Vietnam Posts And Telecommunications Group,14.162.0.0/15,NaN,NaN,NaN,140.292398,NaN,NaN
632309,1.578592e+09,"[1578592189.168852, 1578592189.169105, 1578592...",1174.196899,785,20582.203822,65535.0,82280.0,19552.0,54871.0,445.0,614.0,80.0,210.0,35.0,263.0,14.163.174.30,210.242.149.103,98.015287,114,6,eth:ethertype:ip:tcp,eth:ether

In [10]:
df_filter51_smb.ip_src.tolist()

NameError: name 'df_filter51_smb' is not defined

#### case12

In [ ]:
case12_ips = df_filter51_smb.frame_time_list.tolist()
case12_ips = [list(x) for x in case12_ips]
pickle.dump(obj=case12_ips,file=open(pickle_dir+'case12.pkl','wb'))

In [ ]:
df_filter51_smb.columns

#### case12-1

In [ ]:
case121_ips = df[(df.ip_src=='113.189.231.122')&(df.frame_protocols_max.str.contains('eth:ethertype:ip:tcp:nbss:smb'))].frame_time_list.tolist()
case121_ips = [list(x) for x in case121_ips]
cas121final_ips = []
for li in case121_ips:
    final = set(li) - set(case12_ips[0])
    cas121final_ips.append(list(final))
# case121_ips = set(case121_ips) - set(case12_ips[0])
pickle.dump(obj=cas121final_ips,file=open(pickle_dir+'case12-1.pkl','wb'))
len(cas121final_ips)
# - sㄒ

payload

In [ ]:
case12 = pickle.load(open(pickle_dir+'case12_payload.pkl','rb'))
case12

case12-2

In [29]:
case122_filter1 = df[(df.frame_protocols_max == df_filter51_smb.frame_protocols_max.iloc[0])&(df.tcp_dstport == df_filter51_smb.tcp_dstport.iloc[0])&(df.frame_protocols_common == df_filter51_smb.frame_protocols_common.iloc[0])]
case122_filter2 = case122_filter1[(case122_filter1.tcp_payload_avg<=df_filter51_smb.tcp_payload_avg.max())&(case122_filter1.tcp_payload_avg>=df_filter51_smb.tcp_payload_avg.min())]
case122_filter3 = case122_filter2[(case122_filter2.session_packets<=df_filter51_smb.session_packets.max())&(case122_filter2.session_packets>=df_filter51_smb.session_packets.min())]
case122_filter4 = case122_filter3[(case122_filter3.session_duration<=df_filter51_smb.session_duration.max())&(case122_filter3.session_duration>=df_filter51_smb.session_duration.min())]
case122_filter5 = case122_filter4[(case122_filter4.tcp_payload_size<=df_filter51_smb.tcp_payload_size.max())&(case122_filter4.tcp_payload_size>=df_filter51_smb.tcp_payload_size.min())]
case122_filter6 = case122_filter5[(case122_filter5.frame_len_sum<=df_filter51_smb.frame_len_sum.max())&(case122_filter5.frame_len_sum>=df_filter51_smb.frame_len_sum.min())]
case122_filter6 = case122_filter6[(case122_filter6.isp!='Googlebot')&(case122_filter6.domain != 'vnpt.vn')&(case122_filter6.domain != 'googlebot.com')]
case122_filter6

,frame_time,frame_time_list,session_duration,session_packets,tcp_window_size_value_avg,tcp_window_size_value_max,tcp_payload_size,tcp_hdr_len_sum,tcp_srcport,tcp_dstport,tcp_flags_ack,tcp_flags_fin,tcp_flags_push,tcp_flags_reset,tcp_flags_syn,ip_src,ip_dst,ip_ttl_avg,ip_ttl_max,ip_proto,frame_protocols_common,frame_protocols_max,frame_len_sum,country,city,subdivisions,latitude,longitude,domain,isp,network,udp_length_sum,udp_srcport,udp_dstport,tcp_payload_avg,udp_payload_size,udp_payload_avg
18521,1.578625e+09,"[1578624791.862712, 1578624791.862892, 1578624...",619.007507,446,10319.248879,29200.0,84225.0,10500.0,53851.0,445.0,341.0,35.0,140.0,17.0,123.0,117.6.225.134,210.242.147.71,95.746637,112,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,110449,Vietnam,Hoang Mai,Tinh Bac Giang,21.2500,106.1167,None,Viettel Group,117.6.0.0/15,NaN,NaN,NaN,188.845291,NaN,NaN
30448,1.578592e+09,"[1578592067.718862, 1578592067.719193, 1578592...",1080.856445,643,26358.211509,65535.0,72845.0,16200.0,55185.0,445.0,450.0,85.0,130.0,27.0,249.0,58.27.253.2,210.242.148.253,97.752722,113,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,111491,Pakistan,Karachi,Sindh,24.9043,67.0817,wateen.net,Wateen Telecom,58.27.128.0/17,NaN,NaN,NaN,113.289269,NaN,NaN
71914,1.578648e+09,"[1578647857.413633, 1578647857.414027, 1578647...",558.807251,378,21095.195767,64308.0,71079.0,8672.0,57314.0,445.0,313.0,39.0,91.0,15.0,86.0,113.176.100.12,210.242.146.132,95.891534,114,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,93131,Vietnam,Da Nang,Da Nang,16.0685,108.2215,None,Vietnam Posts And Telecommunications Group,113.176.0.0/16,NaN,NaN,NaN,188.039683,NaN,NaN
103016,1.578600e+09,"[1578600077.892736, 1578600077.892955, 1578600...",789.529236,571,21067.313485,64400.0,79059.0,13780.0,58978.0,445.0,458.0,45.0,175.0,26.0,167.0,49.213.27.28,210.242.146.69,56.488616,64,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,112831,Hong Kong,None,None,22.2500,114.1667,None,SG.GS,49.213.24.0/21,NaN,NaN,NaN,138.457093,NaN,NaN
111767,1.578623e+09,"[1578622857.711833, 1578622857.712123, 1578622...",581.942017,376,25435.808511,64952.0,71955.0,9152.0,62502.0,445.0,272.0,38.0,98.0,20.0,129.0,187.189.26.233,210.242.148.209,97.175532,110,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,94227,Mexico,Mérida,Yucatán,20.9840,-89.6395,totalplay.net,Totalplay,187.189.26.0/23,NaN,NaN,NaN,191.369681,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3527393,1.578654e+09,"[1578653580.716742, 1578653580.717011, 1578653...",778.502930,586,42316.145051,64800.0,75106.0,13992.0,50358.0,445.0,432.0,66.0,145.0,29.0,213.0,1.10.132.97,210.242.147.225,104.394198,244,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,109488,Thailand,Bangkok,Bangkok,13.7366,100.4995,totinternet.net,TOT,1.10.128.0/17,NaN,NaN,NaN,128.167235,NaN,NaN
3528097,1.578668e+09,"[1578667829.021799, 1578667829.022076, 1578667...",1424.456177,917,36109.001091,65535.0,88374.0,21792.0,49380.0,445.0,697.0,102.0,257.0,49.0,302.0,114.27.34.71,210.242.149.112,103.633588,120,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,142124,Taiwan,None,None,23.5000,121.0000,hinet.net,HiNet,114.26.0.0/15,NaN,NaN,NaN,96.372955,NaN,NaN
3566787,1.578626e+09,"[1578625789.007523, 1578625789.009074, 1578625...",649.742676,453,36298.192053,64400.0,71658.0,11076.0,22637.0,445.0,321.0,48.0,121.0,19.0,160.0,117.6.86.139,210.242.148.108,98.072848,111,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,98572,Vietnam,Hanoi,Hanoi,21.0313,105.8516,None,Viettel Group,117.6.0.0/15,NaN,NaN,NaN,158.185430,NaN,NaN
3634519,1.578660e+09,"[1578659767.926574, 1578659767.926801, 1578659...",674.812683,614,40675.361564,65392.0,71981.0,14348.0,59522.0,445.0,468.0,84.0,114.0,23.0,175.0,49.248.36.154,210.242.147.114,95.459283,108,6,eth:ethertype:ip:tcp,eth:ethertype:ip:tcp:nbss:smb,107953,India,Mumbai,Maharashtra,19.0368,73.0158,49-tataidc.co.in,Tata Tele

In [30]:
case122_ips = case122_filter6.frame_time_list.tolist()
case122_ips = [list(x) for x in case122_ips]
pickle.dump(obj=case122_ips,file=open(pickle_dir+"case12-2.pkl",'wb'))
len(case122_ips)

184

#### case13

In [ ]:
df[df.ip_src == '210.109.97.141']

In [ ]:
df[df.frame_time==1578639383.688252] #case13

#### case13-1

In [ ]:
case13_filter1 = df[(df.ip_src == '210.109.97.141')&(df.frame_protocols_max != 'eth:ethertype:ip:tcp:tds:gss-api')]
case13_filter2 = case13_filter1[(case13_filter1.udp_payload_size>0) | (case13_filter1.tcp_payload_size>0)]
case13_filter2

In [ ]:
case13_1 = case13_filter2.frame_time_list.tolist()
case13_1 = [list(x) for x in case13_1]
pickle.dump(obj=case13_1,file=open(pickle_dir+'case13-1.pkl','wb'))

In [ ]:
case13_filter1.ip_proto.value_counts()

In [ ]:
case13_filter1[case13_filter1.ip_proto=="17"]

In [ ]:
1578639383.688252
1578639882.501885

統計

In [4]:
all_sess = df.frame_protocols_max.value_counts().sum()

In [5]:
df.frame_protocols_max.value_counts()#.index.str.contains('smb')
indexes = df.frame_protocols_max.value_counts().index.tolist()
values = df.frame_protocols_max.value_counts().values#.tolist()

In [7]:
smb = 0
http = 0
tds = 0
ssh = 0
bvlc = 0
nbns = 0
ssdp = 0
ntp = 0
rpc = 0
dns = 0
for idx, val in zip(indexes,values):
    if 'smb' in idx:
        smb = smb+val
    if 'http' in idx:
        http = http + val
    if 'tds' in idx:
        tds = tds + val
    if 'ssh' in idx:
        ssh = ssh+val
    if 'bvlc' in idx:
        bvlc = bvlc + val
    if 'nbns' in idx:
        nbns = nbns + val
    if 'ssdp' in idx:
        ssdp = ssdp +val
    if 'ntp' in idx:
        ntp = ntp+val
    if 'rpc' in idx:
        rpc = rpc + val
    if 'dns' in idx:
        dns = dns+val

In [9]:
smb

116091

In [78]:
dns/all_sess

0.0029602591969815034

case14

In [ ]:
glastop_ips = ['61.2.150.64','111.42.67.54','211.230.143.190','116.114.95.142','111.43.223.45',
               '115.61.40.231','42.239.9.38','111.42.66.178','120.71.186.129','111.43.223.54']
glastop_df = df[df.ip_src.isin(glastop_ips)]
glastop_df = glastop_df[(glastop_df.session_packets>1) & (glastop_df.session_duration>1)]
glastop_df = glastop_df[(glastop_df.udp_payload_size>1) | (glastop_df.tcp_payload_size>1)]
glastop_df

In [ ]:
case14_ips = glastop_df.frame_time_list.tolist()
case14_ips = [list(x) for x in case14_ips]
pickle.dump(obj=case14_ips, file=open(pickle_dir+'case14.pkl','wb'))
len(case14_ips)

payload

In [ ]:
case14 = pickle.load(open(pickle_dir+'case14_payload.pkl','rb'))
case14

#### case15

In [ ]:
r = open(pickle_dir+'cowrie_IPs.txt','r')
ips = r.readlines()
ips = [x.split('\n')[0] for x in ips]
ips

In [ ]:
cowrie_df = df[df.ip_src.isin(ips)]
cowrie_df = cowrie_df[(cowrie_df.session_packets>1) & (cowrie_df.session_duration>1)]
cowrie_df = cowrie_df[(cowrie_df.udp_payload_size>1) | (cowrie_df.tcp_payload_size>1)]
cowrie_df_not = cowrie_df[cowrie_df.frame_protocols_max!='eth:ethertype:ip:tcp:ssh']
cowrie_df_not

In [ ]:
not_ssh = cowrie_df_not.frame_time_list.tolist()
not_ssh = [list(x) for x in not_ssh]
pickle.dump(obj=not_ssh,file=open(pickle_dir+'case15_notssh.pkl','wb'))

In [ ]:
cowrie_df = cowrie_df[cowrie_df.frame_protocols_max=='eth:ethertype:ip:tcp:ssh']
ssh = cowrie_df.frame_time_list.tolist()
ssh = [list(x) for x in ssh]
pickle.dump(obj=ssh,file=open(pickle_dir+'case15_ssh.pkl','wb'))
len(ssh)

payload

In [ ]:
case15_ssh = pickle.load(open(pickle_dir+'case15_ssh_payload.pkl','rb'))
len(case15_ssh)

In [ ]:
case15_ssh[0]

In [ ]:
one = df[df.ip_src == '151.30.49.40']
two = df[df.ip_src == '178.92.85.233']
merged = one.append(two)#.to_excel(pickle_dir+'ttc_cases_time.xlsx',index=False)
merged

In [ ]:
def expand_exact_time(li):
    '''
    input: list of time
    output: exact time split by 、
    '''
    li_ = [str(x) for x in li]
    return "、".join(li_)
merged['frame_time_list'] = merged.frame_time_list.map(expand_exact_time)
merged.to_excel(pickle_dir+'ttc_cases_time.xlsx',index=False)
merged